In [1]:
import os
import shutil
import datetime

import random
import pandas as pd
import numpy as np
import pdb

In [2]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.losses import binary_crossentropy, categorical_crossentropy, mean_squared_error
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras import Input

In [3]:
%load_ext tensorboard

In [39]:
keras.backend.clear_session()

with tf.name_scope("Wong"):    
    inputs = Input(shape=(None, 10), name="inputs")
    x = layers.GRU(32, dropout = 0.1, recurrent_dropout=0.5, return_sequences=True, name="GRU_1")(inputs)
    outputs = layers.GRU(10, activation='relu', dropout=0.1, recurrent_dropout=0.5, return_sequences=True, name="GRU_2")(x)
    model = Model(inputs, outputs)

In [40]:
train_data = np.empty((10, 10))
for i in range(train_data.shape[0]):
    train_data[i, :] = np.concatenate((np.random.randint(0, 2, 2), 
                                       np.array([0, 1, 0]), 10 * np.random.rand(5)))
train_data = np.array([train_data, train_data, train_data])
train_data = train_data.astype(np.float32)
train_labels = train_data[:]

In [36]:
print(train_data[:, 1:].shape)

(3, 9, 10)


In [26]:
fn_bce = tf.keras.losses.BinaryCrossentropy()
fn_cce = tf.keras.losses.CategoricalCrossentropy()
fn_mse = tf.keras.losses.MeanSquaredError()

In [41]:
c_nCtg = 2
c_nNum = 5
c_nSeqsLen = 10
c_nBatchLen = 3

def fnBatchLoss(y_true, y_pred):
    #pdb.set_trace()

    fBinLoss = fn_bce(y_true = y_true[:,  :, : c_nCtg], y_pred=y_pred[:, :c_nSeqsLen - 1, : c_nCtg], sample_weight=0.1)
    fCtgLoss = fn_cce(y_true=y_true[:, :, c_nCtg: c_nNum], y_pred=y_pred[:, :c_nSeqsLen - 1, c_nCtg : c_nNum], sample_weight=10)
    fNumLoss = fn_mse(y_true=y_true[:, :, c_nNum : ], y_pred=y_pred[: , :c_nSeqsLen - 1, c_nNum: ])
    
    fLoss = tf.add_n([fBinLoss, fCtgLoss, fNumLoss])
        
    return fLoss


In [17]:
optimizer = tf.optimizers.SGD()

In [42]:
model.compile(loss=fnBatchLoss, optimizer=optimizer)

In [29]:
logdir = './tb/'
if os.path.exists(logdir):
    shutil.rmtree(logdir)

In [20]:
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

In [43]:
model.fit(train_data, train_labels[:, 1:], epochs=3, callbacks=[tensorboard_callback])

Train on 3 samples
Epoch 1/3
3/3 [==============================] - 2s 767ms/sample - loss: 201.7356
Epoch 2/3
3/3 [==============================] - 0s 6ms/sample - loss: 198.3286
Epoch 3/3
3/3 [==============================] - 0s 5ms/sample - loss: nan


In [44]:
%tensorboard --logdir tb --host 0.0.0.0 --port 6008

Reusing TensorBoard on port 6008 (pid 72), started 0:30:27 ago. (Use '!kill 72' to kill it.)